In [1]:
#Setting Working Directory
import os
os.chdir('C:/Users/Zeddrex Navarra/Desktop/Career/Projects/Train Simulation')

In [2]:
#Importing Packages
import pandas as pd
from scipy.stats import norm
import queue

In [3]:
#Exporting Data
train_data = pd.read_csv("train_data.csv").iloc[:,1:6]
mask = train_data["Day"] == "Weekday"
train_data = train_data[mask]

In [26]:
#Global objects

stations = {
    "North Ave": 0,
    "Quezon Ave": 1.2,
    "GMA Kamuning": 2.2,
    "Cubao": 4.1,
    "Santolan": 5.6,
    "Ortigas": 7.9,
    "Shaw Blvd": 8.7,
    "Boni Ave": 9.7,
    "Guadalupe": 10.5,
    "Buendia": 12.5,
    "Ayala Ave": 13.45,
    "Magallanes": 14.65,
    "Taft": 16.7,
    "End of line": 100
}

occupation = {
    "North Ave": "Not occupied",
    "Quezon Ave": "Not occupied",
    "GMA Kamuning": "Not occupied",
    "Cubao": "Not occupied",
    "Santolan": "Not occupied",
    "Ortigas": "Not occupied",
    "Shaw Blvd": "Not occupied",
    "Boni Ave": "Not occupied",
    "Guadalupe": "Not occupied",
    "Buendia": "Not occupied",
    "Ayala Ave": "Not occupied",
    "Magallanes": "Not occupied",
    "Taft": "Not occupied",
    "End of line": "Not occupied"
}

alighting = {
    "North Ave": 0,
    "Quezon Ave": 0.1,
    "GMA Kamuning": 0.2,
    "Cubao": 0.6,
    "Santolan": 0.1,
    "Ortigas": 0.3,
    "Shaw Blvd": 0.1,
    "Boni Ave": 0.2,
    "Guadalupe": 0.3,
    "Buendia": 0.1,
    "Ayala Ave": 0.4,
    "Magallanes": 0.3,
    "Taft": 1
}

stations_list = list(stations.keys())

[1.2,1,1.9,1.5,2.3,0.8,1,0.8,2,0.95,1.2,2.05]
cum_dist = [1.2,2.2,4.1,5.6,7.9,8.7,9.7,10.5,12.5,13.45,14.65,16.7]
distance_over_time = 0.75

trains = ["t1", "t2", "t3", "t4", "t5", "t6", "t7", "t8", "t9", "t10"]
line = [] #this is where the train tracking happens
pila = {} #this is where the distances are updated
next_station = {} #target station of the train
current_station = {} #current station of the train
parking = {} #to set the parking time of a train

In [5]:
#Train operations
i=0
while i < 1200:
    
    if i%10 == 0:

        if ((occupation["North Ave"] == "Not occupied") and (len(trains) != 0)):
            #Deploys a train every 10 mins if there are trains to deploy and North Ave does not have a train parked
            train = trains.pop(0)
            line.append(train)
            pila[train] = 0
            current_station[train] = "North Ave"
            next_station[train] = "Quezon Ave"
            parking[train] = 2
            occupation["North Ave"] = "Occupied"
            print(f"Train {train} is at North Ave at  time {i}")          
    
    for t in list(line):

        if parking[t] > 0:
            parking[t] = parking[t]-1
            print(f"Train {t} is waiting at station {current_station[t]} for {parking[t]} minutes, current time {i}")
            continue

        if parking[t] <= 0 and occupation[next_station[t]] == "Not occupied":
            occupation[current_station[t]] = "Not occupied"
            target_position = stations[next_station[t]]
            new_position = pila[t] + distance_over_time
            
            if (target_position <= new_position):
                pila[t] = target_position
                current_station[t] = next_station[t]
                occupation[current_station[t]] = "Occupied"
                next_station[t] = stations_list[1 + stations_list.index(next_station[t])]
                parking[t] = 2
                print(f"Train {t} has arrived at {current_station[t]} at time {i}, the next station is {next_station[t]}")
                print([station for station, status in occupation.items() if status == "Occupied"])
                #break
            else:
                pila[t] = new_position

            if new_position > 16.7:
                occupation["Taft"] = "Not occupied"
                occupation["End of line"] = "Not occupied"
                trains.append(t)
                line.pop(0),
                pila.pop(t),
                next_station.pop(t)
                parking.pop(t)
                current_station.pop(t)
                print(f"Train {t} will be returning to the terminal station")
    
    i+=1      

    #print(occupation)

Train t1 is at North Ave at  time 0
Train t1 is waiting at station North Ave for 1 minutes, current time 0
Train t1 is waiting at station North Ave for 0 minutes, current time 1
Train t1 has arrived at Quezon Ave at time 3, the next station is GMA Kamuning
['Quezon Ave']
Train t1 is waiting at station Quezon Ave for 1 minutes, current time 4
Train t1 is waiting at station Quezon Ave for 0 minutes, current time 5
Train t1 has arrived at GMA Kamuning at time 7, the next station is Cubao
['GMA Kamuning']
Train t1 is waiting at station GMA Kamuning for 1 minutes, current time 8
Train t1 is waiting at station GMA Kamuning for 0 minutes, current time 9
Train t2 is at North Ave at  time 10
Train t2 is waiting at station North Ave for 1 minutes, current time 10
Train t2 is waiting at station North Ave for 0 minutes, current time 11
Train t1 has arrived at Cubao at time 12, the next station is Santolan
['North Ave', 'Cubao']
Train t1 is waiting at station Cubao for 1 minutes, current time 13
Tr

In [6]:
#Passenger Generation

station_loop = stations_list[:-1]
count = 0
que = {}
passenger_data = {}

for station in station_loop:
    
    que = {}
    print(station)
    sample = train_data[train_data["Station"] == station]

    for i in range(1200):

        try:    
            j = 4 + (i//60)
            if i%5 == 0:
                loc = sample[sample["Hour"] == j]["Mean"].iloc[0]
                scale = sample[sample["Hour"] == j]["Variance"].iloc[0]

                passenger = (norm.rvs(loc,scale,1,random_state = i)//12).tolist()[0]
                
                if passenger < 0:
                    passenger = 0
                else:
                    passenger = int(passenger)
                
                que[i] = passenger
                count += 1
                
        except:
            continue
    
    passenger_data[station] = que

    print(count)  

North Ave
240
Quezon Ave
480
GMA Kamuning
720
Cubao
960
Santolan
1200
Ortigas
1440
Shaw Blvd
1680
Boni Ave
1920
Guadalupe
2160
Buendia
2400
Ayala Ave
2640
Magallanes
2880
Taft
3120


In [7]:
#Passenger boarding
station_queue = {}
capacity = {}
i=0


while i<1201:

    if i%5 == 0:

        try:

            for station in station_loop:
                
                if station not in station_queue:
                    station_queue[station] = {}

                station_queue[station][i] = passenger_data[station][i]

        except:
            break

    if i%60 == 0:

        for station in station_loop:
            
            if occupation[station] == "Occupied":
                
                capacity[station] = sum(station_queue[station])
                total = sum(station_queue[station].values())
                print(f"{station} has {total} passengers")

            else:
                continue
        
    
    

    i+=1

#print(capacity)
print(sum(station_queue["North Ave"]))

GMA Kamuning has 5 passengers
Buendia has 0 passengers
Magallanes has 0 passengers
GMA Kamuning has 74 passengers
Buendia has 5 passengers
Magallanes has 24 passengers
GMA Kamuning has 447 passengers
Buendia has 79 passengers
Magallanes has 414 passengers
GMA Kamuning has 1036 passengers
Buendia has 292 passengers
Magallanes has 1334 passengers
GMA Kamuning has 1714 passengers
Buendia has 578 passengers
Magallanes has 2537 passengers
GMA Kamuning has 2329 passengers
Buendia has 845 passengers
Magallanes has 3629 passengers
GMA Kamuning has 2959 passengers
Buendia has 1119 passengers
Magallanes has 4647 passengers
GMA Kamuning has 3513 passengers
Buendia has 1425 passengers
Magallanes has 5485 passengers
GMA Kamuning has 4089 passengers
Buendia has 1760 passengers
Magallanes has 6323 passengers
GMA Kamuning has 4721 passengers
Buendia has 2154 passengers
Magallanes has 7292 passengers
GMA Kamuning has 5244 passengers
Buendia has 2534 passengers
Magallanes has 8191 passengers
GMA Kamunin

In [27]:
#Passenger boarding mechanics
train_capacity = {}
station_queue = {}
keys_for_removal = {}
wait_time = []
i=0

while i < 1201:
    
    if i%5 == 0:

        try:

            for station in station_loop:
                
                if station not in station_queue:
                    station_queue[station] = {}

                station_queue[station][i] = passenger_data[station][i]

        except:

            break
    
    if i%10 == 0:

        if ((occupation["North Ave"] == "Not occupied") and (len(trains) != 0)):
            #Deploys a train every 10 mins if there are trains to deploy and North Ave does not have a train parked
            train = trains.pop(0)
            line.append(train)
            pila[train] = 0
            current_station[train] = "North Ave"
            next_station[train] = "Quezon Ave"
            parking[train] = 2
            occupation["North Ave"] = "Occupied"

            keys_for_removal[train] = station_queue["North Ave"].copy()
            train_capacity[train] = sum(station_queue["North Ave"].copy().values())

            for pops in list(keys_for_removal[train].keys()):

                wait_time.append((i-pops,station_queue[current_station[train]][pops]))
                station_queue[current_station[train]].pop(pops)

            print(f"Train {train} is at North Ave at  time {i}, the next station is {next_station[train]}")  
            print(f"Train {train} has {train_capacity[train]} passengers")       

    for t in list(line):

        if parking[t] > 0:
            parking[t] = parking[t]-1
            #print(f"Train {t} is waiting at station {current_station[t]} for {parking[t]} minutes, current time {i}")
            continue

        if parking[t] <= 0 and occupation[next_station[t]] == "Not occupied":

            occupation[current_station[t]] = "Not occupied"
            target_position = stations[next_station[t]]
            new_position = pila[t] + distance_over_time
            
            if (target_position <= new_position):

                #Distance variable
                pila[t] = target_position

                #Station tracker
                current_station[t] = next_station[t]
                occupation[current_station[t]] = "Occupied"
                next_station[t] = stations_list[1 + stations_list.index(next_station[t])]

                #Passenger tracker
                train_capacity[t] = int((train_capacity[t])*(1-alighting[current_station[t]])) + sum(station_queue[current_station[t]].copy().values())
                keys_for_removal[t] = station_queue[current_station[t]].copy()

                for pops in list(keys_for_removal[t].keys()):

                    wait_time.append((i-pops,station_queue[current_station[t]][pops]))
                    station_queue[current_station[t]].pop(pops)
                
                #Parking timer
                parking[t] = 2

                #Diagnostic messages
                print(f"Train {t} has arrived at {current_station[t]} at time {i}, the next station is {next_station[t]}")
                print([station for station, status in occupation.items() if status == "Occupied"])
                print(f"Train {t} has {train_capacity[t]} passengers")
                #break

            else:

                pila[t] = new_position

            if new_position > 16.7:

                occupation["Taft"] = "Not occupied"
                occupation["End of line"] = "Not occupied"
                trains.append(t)
                line.pop(0),
                pila.pop(t),
                next_station.pop(t)
                parking.pop(t)
                current_station.pop(t)
                print(f"Train {t} will be returning to the terminal station")
    
    i+=1      

Train t1 is at North Ave at  time 0, the next station is Quezon Ave
Train t1 has 68 passengers
Train t1 has arrived at Quezon Ave at time 3, the next station is GMA Kamuning
['Quezon Ave']
Train t1 has 75 passengers
Train t1 has arrived at GMA Kamuning at time 7, the next station is Cubao
['GMA Kamuning']
Train t1 has 70 passengers
Train t2 is at North Ave at  time 10, the next station is Quezon Ave
Train t2 has 111 passengers
Train t1 has arrived at Cubao at time 12, the next station is Santolan
['North Ave', 'Cubao']
Train t1 has 93 passengers
Train t2 has arrived at Quezon Ave at time 13, the next station is GMA Kamuning
['Quezon Ave', 'Cubao']
Train t2 has 124 passengers
Train t1 has arrived at Santolan at time 16, the next station is Ortigas
['Quezon Ave', 'Santolan']
Train t1 has 83 passengers
Train t2 has arrived at GMA Kamuning at time 17, the next station is Cubao
['GMA Kamuning', 'Santolan']
Train t2 has 108 passengers
Train t3 is at North Ave at  time 20, the next station is

In [83]:
#Testing module
place = "GMA Kamuning"
holder = 0
passenger_data["Quezon Ave"]

{0: 14,
 5: 12,
 10: 13,
 15: 12,
 20: 13,
 25: 12,
 30: 11,
 35: 10,
 40: 12,
 45: 12,
 50: 11,
 55: 11,
 60: 54,
 65: 58,
 70: 118,
 75: 62,
 80: 156,
 85: 94,
 90: 79,
 95: 64,
 100: 25,
 105: 79,
 110: 99,
 115: 110,
 120: 168,
 125: 69,
 130: 101,
 135: 3,
 140: 96,
 145: 54,
 150: 123,
 155: 223,
 160: 92,
 165: 198,
 170: 154,
 175: 76,
 180: 65,
 185: 123,
 190: 164,
 195: 181,
 200: 30,
 205: 125,
 210: 102,
 215: 118,
 220: 38,
 225: 139,
 230: 175,
 235: 123,
 240: 85,
 245: 103,
 250: 79,
 255: 38,
 260: 108,
 265: 214,
 270: 164,
 275: 195,
 280: 96,
 285: 175,
 290: 67,
 295: 38,
 300: 47,
 305: 120,
 310: 135,
 315: 47,
 320: 69,
 325: 192,
 330: 53,
 335: 132,
 340: 22,
 345: 176,
 350: 186,
 355: 100,
 360: 135,
 365: 116,
 370: 60,
 375: 109,
 380: 132,
 385: 103,
 390: 116,
 395: 62,
 400: 71,
 405: 73,
 410: 121,
 415: 97,
 420: 99,
 425: 90,
 430: 126,
 435: 88,
 440: 104,
 445: 97,
 450: 129,
 455: 63,
 460: 122,
 465: 124,
 470: 114,
 475: 52,
 480: 98,
 485: 102

In [87]:
len(wait_time)

3112

In [12]:
passenger_data["Quezon Ave"]

2

In [23]:
int(3.9)

3